In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285387 sha256=8da840f4b498258e8538fe9e23b1043dbf34393b7988a1886f5a5d3d525e0e67
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [35]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import regexp_replace, when, col

In [36]:
spark = SparkSession.builder \
    .appName("data_clean_salesorderdetail") \
    .getOrCreate()

In [52]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType, TimestampType
from pyspark.sql.functions import regexp_replace

spark = SparkSession.builder.appName("DDLSchemaConversion").getOrCreate()

# Step 1: Create schema based on DDL
schema = StructType([
    StructField("SalesOrderID", IntegerType()),
    StructField("SalesOrderDetailID", IntegerType(), nullable=False),
    StructField("CarrierTrackingNumber", StringType()),
    StructField("OrderQty", IntegerType()),
    StructField("ProductID", IntegerType()),
    StructField("SpecialOfferID", IntegerType()),
    StructField("UnitPrice",StringType()),
    StructField("UnitPriceDiscount", StringType()),
    StructField("LineTotal", FloatType()),
    StructField("rowguid", StringType()),
    StructField("ModifiedDate", TimestampType())
])

# Step 2: Read CSV with enforced schema
df = spark.read.csv('./raw_data/Sales.SalesOrderDetail.csv', schema=schema, sep=';', header=True)

# Step 3: Perform regex operations and other transformations
# Remove all "null" cases in upper and lower case
df = df.withColumn("CarrierTrackingNumber", regexp_replace(df["CarrierTrackingNumber"], "(?i)NULL", ""))
df = df.withColumn("rowguid", regexp_replace(df["rowguid"], "(?i)NULL", ""))

# Substitute commas for dots for specific columns
df = df.withColumn("UnitPrice", regexp_replace(df["UnitPrice"], ",", "."))
df = df.withColumn("UnitPriceDiscount", regexp_replace(df["UnitPriceDiscount"], ",", "."))

df.show()


+------------+------------------+---------------------+--------+---------+--------------+---------+-----------------+---------+--------------------+-------------------+
|SalesOrderID|SalesOrderDetailID|CarrierTrackingNumber|OrderQty|ProductID|SpecialOfferID|UnitPrice|UnitPriceDiscount|LineTotal|             rowguid|       ModifiedDate|
+------------+------------------+---------------------+--------+---------+--------------+---------+-----------------+---------+--------------------+-------------------+
|       43659|                 1|         4911-403C-98|       1|      776|             1| 2024.994|             0.00| 2024.994|B207C96D-D9E6-402...|2011-05-31 00:00:00|
|       43659|                 2|         4911-403C-98|       3|      777|             1| 2024.994|             0.00| 6074.982|7ABB600D-1E77-41B...|2011-05-31 00:00:00|
|       43659|                 3|         4911-403C-98|       1|      778|             1| 2024.994|             0.00| 2024.994|475CF8C6-49F6-486...|2011-05

In [53]:
# Cast "UnitPrice" and "UnitPriceDiscount" to float
df = df.withColumn("UnitPrice", col("UnitPrice").cast(FloatType()))
df = df.withColumn("UnitPriceDiscount", col("UnitPriceDiscount").cast(FloatType()))

df.show()

+------------+------------------+---------------------+--------+---------+--------------+---------+-----------------+---------+--------------------+-------------------+
|SalesOrderID|SalesOrderDetailID|CarrierTrackingNumber|OrderQty|ProductID|SpecialOfferID|UnitPrice|UnitPriceDiscount|LineTotal|             rowguid|       ModifiedDate|
+------------+------------------+---------------------+--------+---------+--------------+---------+-----------------+---------+--------------------+-------------------+
|       43659|                 1|         4911-403C-98|       1|      776|             1| 2024.994|              0.0| 2024.994|B207C96D-D9E6-402...|2011-05-31 00:00:00|
|       43659|                 2|         4911-403C-98|       3|      777|             1| 2024.994|              0.0| 6074.982|7ABB600D-1E77-41B...|2011-05-31 00:00:00|
|       43659|                 3|         4911-403C-98|       1|      778|             1| 2024.994|              0.0| 2024.994|475CF8C6-49F6-486...|2011-05

In [54]:
# Writing the transformed dataframe to a new CSV file
df.coalesce(1).write.csv('table_salesorderdetail.csv', header=True, mode='overwrite')

In [55]:
spark.stop()